<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [331]:
# импорт необходимых библиотек, требующихся для выполнения всех заданий
# выключение предупреждений (предупреждение о том, что pandas поддерживает только SQLAlchemy connectable (engine/connection) или URL-адрес строки базы данных или подключение sqlite3 DBAPI2)
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [1]:
DBNAME = 'KA'
USER = 'PI'
PASSWORD = 'BA'
HOST = 'RA'
PORT = 'не скажу'

In [333]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [334]:
# текст запроса
query_3_1 = f'''SELECT 
                    COUNT(id) vacancies
                FROM 
                    vacancies
'''

In [335]:
# результат запроса
quantity_vacancies = pd.read_sql_query(query_3_1, connection)
quantity_vacancies

,vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [336]:
# текст запроса
query_3_2 = f'''SELECT 
                    COUNT(id) employers
                FROM 
                    employers
             '''

In [337]:
# результат запроса
quantity_employers = pd.read_sql_query(query_3_2, connection)
quantity_employers

,employers
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [338]:
# текст запроса
query_3_3 = f'''SELECT 
                    COUNT(id) areas
                FROM 
                    areas
                '''

In [339]:
# результат запроса
quantity_areas = pd.read_sql_query(query_3_3, connection)
quantity_areas

,areas
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [340]:
# текст запроса
query_3_4 = f'''SELECT
                    COUNT(id) industries
                FROM
                    industries
                '''

In [341]:
# результат запроса
quantity_industries=pd.read_sql_query(query_3_4, connection)
quantity_industries

,industries
0,294


***

выводы по предварительному анализу данных.

- количество вакансий.
- количество работодателей.
- количество регеонов.
- сфер деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [342]:
# текст запроса
query_4_1 = f'''SELECT
                    a.name,
                    COUNT(v.id) cnt -- подстчет количества по id вакансий
                FROM
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- получаем названия регионов для таблицы с вакансиями
                GROUP BY a.name -- группируем по регионам
                ORDER BY cnt DESC 
             '''

In [343]:
# результат запроса
area_vacancies = pd.read_sql_query(query_4_1, connection)
area_vacancies.head()

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [344]:
# текст запроса
query_4_2 = f'''SELECT
                    COUNT(id)
                FROM
                    vacancies 
                WHERE
                    salary_from IS NOT NULL OR salary_to IS NOT NULL -- заполнено хотя бы одно поле с зарплатой 
             '''

In [345]:
# результат запроса
salary_not_empty = pd.read_sql_query(query_4_2, connection)
salary_not_empty

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [346]:
# текст запроса
query_4_3 = f'''SELECT
                    ROUND(AVG(salary_from))::int avg_salary_from, -- уберем нули приведя к типу int
                    ROUND(AVG(salary_to))::int avg_salary_to 
                FROM
                    vacancies
             '''

In [347]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_salary

,avg_salary_from,avg_salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [348]:
# текст запроса
query_4_4 = f'''WITH sched_employment AS ( --  основная таблица
                    SELECT
                        schedule, 
                        employment,                     
                        COUNT(id) vacancies
                    FROM
                        vacancies 
                    GROUP BY schedule, employment -- группировка по графику, а внутри по типу занятости
                )
                SELECT
                    schedule, 
                    employment,                     
                    vacancies,
                    ROUND(vacancies/(SELECT SUM(vacancies) FROM sched_employment)*100, 5) percent -- вычисляем процент от общего количества вакансий, сильно округлить неполучается петеряются числа около нуля
                FROM
                    sched_employment
                ORDER BY vacancies DESC -- сортируем, как указано
             '''

In [349]:
# результат запроса
sched_employment = pd.read_sql_query(query_4_4, connection)
sched_employment.head()

,schedule,employment,vacancies,percent
0,Полный день,Полная занятость,35367,71.88853
1,Удаленная работа,Полная занятость,7802,15.85869
2,Гибкий график,Полная занятость,1593,3.23800
3,Удаленная работа,Частичная занятость,1312,2.66683
4,Сменный график,Полная занятость,940,1.91069


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [350]:
# текст запроса
query_4_5 = f'''WITH exp_how_many AS ( -- это основная таблица
                    SELECT
                        experience,
                        COUNT(id) quantity
                    FROM
                        vacancies
                    GROUP BY experience -- группировка по опыту
                )
                SELECT
                    experience,
                    quantity,
                    ROUND(quantity/(SELECT SUM(quantity) FROM exp_how_many)*100, 1) percent -- вычисляем процент от общего количества вакансий, округляем до одной десятой
                FROM
                    exp_how_many
                ORDER BY quantity -- сортируем в порядке возрастания
            '''

In [351]:
# результат запроса
experience_vacancies = pd.read_sql_query(query_4_5, connection)
experience_vacancies

,experience,quantity,percent
0,Более 6 лет,1337,2.7
1,Нет опыта,7197,14.6
2,От 3 до 6 лет,14511,29.5
3,От 1 года до 3 лет,26152,53.2


***

выводы по детальному анализу вакансий
- Распределение вакансий по регеонам, нам показало, прямо коррелирует с количеством населения, административным статусом, количесством экономических субъектов (компаний, предприятий, научных учреждений и т.д.) в данном регеоне. В первую сотню по количеству вакансий входят административные субъекты РФ , а также сталицы и крупные города СНГ.
    
- Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно (правда половина работодателей вообще не указала данных о зарплате).
    
- Распределение вакансий для каждого сочетания типа рабочего графика и типа трудоустройства показывает явное предпочтение работодателя к тотальному физическому контролю над своим наемным персоналом. Более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится лишь около 19 %. Вахтовый метод где-то 0.8 %.
    
- Требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. Для новичков остается всего 14.6% вакансий , где опыт не указан. Но как мы видим чуть более 50% вакансий требую хоть какой то опыт (от 1 года до 3 лет).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [352]:
# текст запроса
query_5_1 = f'''WITH employers AS ( -- тут создадим таблицу с количеством вакансий у каждого работодателя, с сортировкой по убыванию
                    SELECT
                        e.name,
                        COUNT(v.id) quantity
                    FROM
                        vacancies v
                        LEFT JOIN employers e ON v.employer_id = e.id -- получаем названия работодателей
                    GROUP BY e.id
                    ORDER BY quantity DESC
                )
                (SELECT -- тут возьмем 1 строку из созданной таблицы
                     name,
                     quantity,
                     1 place
                 FROM
                     employers
                 LIMIT 1)
                UNION -- объединим две нужные строки в новую табличку
                (SELECT -- тут возьмем 5 строку из созданной таблицы
                     name,
                     quantity,
                     5
                 FROM
                     employers
                 OFFSET 4
                 LIMIT 1)
                 ORDER BY place -- отсортируем 
            '''

In [353]:
# результат запроса
employer_vacancies = pd.read_sql_query(query_5_1, connection)
employer_vacancies

,name,quantity,place
0,Яндекс,1933,1
1,Газпром нефть,331,5


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [354]:
# текст запроса
query_5_2 = f'''SELECT
                    a.name,
                    vacan.vacan_quantity,
                    employ.employ_quantity
                FROM areas a -- присоединяем слева к таблице с регионами таблицы vacan и employ по ID регионов, получим для каждого региона количество вакансий и работодателей
                     LEFT JOIN (SELECT 
                                    a.id id,
                                    COUNT(v.id) vacan_quantity
                                FROM
                                    vacancies v 
                                    LEFT JOIN areas a ON v.area_id = a.id
                                GROUP BY a.id -- таблица vacan с названиями регионов и количеством вакансий по каждому
                                ) vacan ON a.id = vacan.id
                     LEFT JOIN (SELECT
                                    a.id id,
                                    COUNT(e.id) employ_quantity
                                FROM
                                    employers e 
                                    LEFT JOIN areas a ON e.area = a.id
                                GROUP BY a.id -- таблица employ с названиями регионов и количеством работодателей в них
                                ) employ ON a.id = employ.id
                WHERE employ_quantity IS NOT NULL -- оставим только те регионы, в которых есть хотя бы один работодатель
                ORDER BY vacan_quantity DESC, employ_quantity DESC -- сортируем по количеству вакансий по убыванию, при этом все отсутствующие будут сверху, а после этой сортировки сортируем еще по количеству работодателей по убыванию и тогда в первой строке получим требуемый регион
             '''

In [355]:
# результат запроса
area_vacan_employ = pd.read_sql_query(query_5_2, connection)
area_vacan_employ.head()

,name,vacan_quantity,employ_quantity
0,Россия,NaN,410
1,Казахстан,NaN,207
2,Московская область,NaN,75
3,Краснодарский край,NaN,19
4,Ростовская область,NaN,18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [356]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name,
                    COUNT(DISTINCT v.area_id) quantity -- количество регионов для каждого работодателя
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- добавляем слева для названия работодателя
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY quantity DESC
            '''

In [357]:
# результат запроса
employ_vacan_area = pd.read_sql_query(query_5_3, connection)
employ_vacan_area.head()

,name,quantity
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [358]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT(*)
                FROM
                    employers e
                    FULL JOIN employers_industries e_i ON e.id = e_i.employer_id -- полное соединение, получаем записи (строки) для каждого работодателя со всеми id сферы деятельности (построчно) или одну запись с "пустышкой" на месте id сферы деятельности (если для id работодателя нет ни одной записи в таблице employers_industries)
                WHERE 
                    e_i.industry_id IS NULL -- оставляем только записи с пустыми полями id сферы деятельности 
            '''

In [359]:
# результат запроса
employ_industies_empty = pd.read_sql_query(query_5_4, connection)
employ_industies_empty

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [360]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name
                FROM
                    employers_industries e_i
                    LEFT JOIN employers e ON e_i.employer_id = e.id -- добавляем названия и id работодателей
                GROUP BY e.id -- группируем по id работодателя
                HAVING COUNT(e_i.industry_id) = 4 -- оставляем только тех работодателей, у которых 4 записи (4 сферы деятельности)
                ORDER BY name -- сортировка по алфавиту
                OFFSET 2 -- отступ на 2 строки
                LIMIT 1 -- вывод первой после отступа
            '''

In [361]:
# результат запроса
third_employ_4_industies = pd.read_sql_query(query_5_5, connection)
third_employ_4_industies

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [362]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(DISTINCT employer_id) quantity -- подсчет количества уникальных id работодателей
                FROM
                    employers_industries e_i
                    LEFT JOIN industries i ON e_i.industry_id = i.id -- добавляем сферы деятельности
                WHERE i.name ~ 'Разработка программного обеспечения' 
             '''

In [363]:
# результат запроса
employ_certain_industry = pd.read_sql_query(query_5_6, connection)
employ_certain_industry

,quantity
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [364]:
# код для получения списка городов-милионников

# импорт необходимых библиотек
import requests
from bs4 import BeautifulSoup

# извлечение данных со страницы по указанному адресу
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url).text

# создание объекта типа BeautifulSoup с использованием парсера lxml
# потом методом select получаем содержимое нужной таблицы с городами-миллионниками - это список
# дальше для 1 элемента списка точно также получаем список содержимого для тегов a с атрибутами href и title
soup = BeautifulSoup(response, 'html.parser')
tab_content = soup.select('#mw-content-text > div.mw-parser-output > table.standard.sortable > tbody')
cities = tab_content[0].select('a[href][title]')

# в результате получено искомое, но есть два лишних элемента - две ссылки в названиях столбцов таблицы, 
# удовлетворяющих условию, нужно их удалить
# подготовим новый пустой список
city_list = []

# в цикле проверим все элементы имеющегося списка на условия - 
# не больше двух слов и первая буква заглавная
for i in cities:
    elem = i.text
    if (len(elem.split()) <= 2) & (elem[0].isupper()):
        city_list.append(elem)

# преобразуем список в кортеж и выведем
city_list = tuple(city_list)

print(city_list)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [365]:
# текст запроса
query_5_7 = f'''WITH tab AS (
                SELECT
                    a.name,
                    COUNT(*)::int cnt
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id
                    LEFT JOIN areas a ON v.area_id = a.id -- в таблицу вакансий добавляем названия работодателей и регионов, методом присоединения слева
                WHERE 
                    e.name LIKE 'Яндекс' -- условие: только Яндекс
                    AND a.name IN {city_list} -- условие: только из списка миллионников точнее кортежа
                GROUP BY a.id
                ) -- cоздаем таблицу tab с регионами из заданного списка (города-миллионники) и количеством вакансий для каждого, исключительно для Яндекса
                
                SELECT 
                    *
                FROM
                    tab
                UNION -- добавляем к имеющейся таблице tab общее количество вакансий по всем городам-миллионникам из нее же
                SELECT
                    'Total',
                    SUM(cnt) -- общая сумма вакансий
                FROM
                    tab
                ORDER BY cnt -- сортируем по количеству вакансий по возрастанию
             '''

In [366]:
# результат запроса
yandex_million_areas = pd.read_sql_query(query_5_7, connection)
yandex_million_areas.tail()

,name,cnt
12,Нижний Новгород,36
13,Екатеринбург,39
14,Санкт-Петербург,42
15,Москва,54
16,Total,485


***

выводы по анализу работодателей
- работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть.
- количество работодателей и вакансий для каждого региона мы тоже уже видели ранее, по отдельности. Оказалось,
    что 186 регионов без вакансий, но об этом тоже говорили. Здесь страны, и другие составные субъекты, отдельные части
    которых присутствуют в перечне регионов. Плюс маленькие регионы, без текущей потребности в ИТ специалистах.
- компанию 2ГИС указала 4 сферы деятельности.
- посмотрим количество вакансий Яндекса по городам-миллионникам, которых на момент было 16. Всего 485 вакансий, что составляет четверть от всех вакансий Яндекса.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [367]:
# текст запроса
query_6_1 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies v
                WHERE 
                    lower(v.name) ~* 'data' OR lower(v.name) ~* 'данн' -- приводим значение поля к нижнему регистру и проверяем наличие двух заданных эелементов в нем
             '''

In [368]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [369]:
# текст запроса
query_6_2 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (lower(name) ~* 'junior' OR
                    experience ='Нет опыта' OR
                    employment = 'Стажировка') -- применение условия для уровня 'junior': наличие элемента 'junior' в поле name (причем в любом регистре, для этого приводим название вакансии к нижнему регистру), для опыта и занятости соответствие заданным значениям
             '''

In [370]:
# результат запроса
junior_vacancies = pd.read_sql_query(query_6_2, connection)
junior_vacancies

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [371]:
# текст запроса
query_6_3 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~* 'SQL' OR
                    key_skills ~* 'postgres') -- соответствие условию наличия двух ключевых навыков, причем в любом месте значения поля (с учетом регистра)
             '''

In [372]:
# результат запроса
sql_vacancies = pd.read_sql_query(query_6_3, connection)
sql_vacancies

,count
0,201


Вроде все правильно а ответ не верный

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [373]:
# текст запроса
query_6_4 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~ 'Python')-- соответствие условию наличия ключевого навыка, причем в любом месте значения поля (с учетом регистра)
             '''

In [374]:
# результат запроса
python_vacancies = pd.read_sql_query(query_6_4, connection)
python_vacancies

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [375]:
# текст запроса
query_6_5 = f'''SELECT
                    ROUND(AVG(length(key_skills) - length(replace(key_skills, '\t', '')) + 1), 2) skills -- считаем количество символов в строке и вычитаем количество символов в строке, где удалили символы разделители '\', т.о. вычисляем количество символов разделитетелей, да + 1 (так как разделитетелей на 1 меньше чем разделяемых элементов), ну и считаем среднее с округлением до 2 знаков
                FROM
                    vacancies
                WHERE
                    name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*' --  условие соответствия вакансии дата-сайентиста
             '''

In [376]:
# результат запроса
avg_skills = pd.read_sql_query(query_6_5, connection)
avg_skills

,skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [377]:
# текст запроса
query_6_6 = f'''SELECT
                    experience,
                    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::int avg_salary -- слагаемые дадут нам или две минимальных или две максимальных, ну или минимальную и максимальную зарплаты, да на 2 поделим, вот и будет или средняя или просто минимум или максимум
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') --  для вакансий именно дата-сайентистов
                    AND
                    (salary_from IS NOT NULL OR salary_to IS NOT NULL) -- условие наличия данных хотя бы для минимальной или максимальнной зарплаты
                GROUP BY experience -- сгруппируем по опыту
                ORDER BY avg_salary DESC -- сортируем по убыванию 
                '''

In [378]:
# результат запроса
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

,experience,avg_salary
0,От 3 до 6 лет,243115
1,От 1 года до 3 лет,139675
2,Нет опыта,74643


In [379]:
# Сделаем запрос на работодателей которые ищут DS
query_6_e = f'''SELECT 
                    e.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого работодателя 
                FROM 
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- присоединяем для получения названий 
                WHERE
                    (v.name ~* 'data scientist' OR 
                    v.name ~* 'data science' OR 
                    v.name ~* 'исследователь данных' OR
                    (v.name ~ 'ML' AND v.name !~* 'HTML') OR
                    v.name ~* 'machine learning' OR
                    v.name ~* 'машинн.*обучен.*') -- условие соответствия вакансии дата-сайентиста
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY vacancies DESC -- сортировка по убыванию
             '''

In [380]:
# результат запроса
employers_ds = pd.read_sql_query(query_6_e, connection)
employers_ds.head(10)

,name,vacancies
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


***

выводы по предметному анализу
-  1771 вакансий имеет отношение к данным. То есть 3.6 % от всего количества.
-  вакансий для джунов очень мало, это значит что есть проблема огромной конкуренции.
-  но если ты владеешь дополнительными языками (SQL или postgres) то уже вакансиий будет больше на рассмотрение. чем больше знаешь тем ты более востребовательнее
-  среди работодателей которые ищут DS оказался на первом месте сбербанк 

# Общий вывод по проекту

Анализ всех вакансий:
- общее количество вакансий 49197, общее количество работодателей 23501, а вот с вакансиями только 14906. Т.о. среднее количество вакансий у работодателя: 3.3 (рассматриваем работодателей с вакансиями).
- распределение вакансий по регионам, прямо коррелирует с «масштабом региона». В первую сотню по количеству вакансий входят административные центры субъектов Российской Федерации, крупнейшие города помимо них, а также столицы и крупные города стран СНГ. Так на долю Москвы приходится почти 11 % от общего количества вакансий, доля Санкт-Петербурга почти 6 %. На третьем месте Минск с более, чем 4 %.
- более половины вакансий - 51 %, не имеют каких-либо указаний на предлагаемую заработную плату, не минимума ни максимума. В 21 % имеется полная информация о предложении работодателя по зарплате. В отношении только минимума определились в 23 % вакансий, а максимума в 5 % вакансий. Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно. Минимум нижней и верхней границ предлагаемой зарплаты определить не просто, т.к. очевидно, что в минимальных значениях пропущены нули или разряд "тысячи". Исходя из здравого смысла следует ориентироваться на минимальные значения от 15 - 20 тыс. рублей. Максимум указан для обеих границ один – 1000000 руб. Визуализация показывает, что основная часть вакансий имеет зарплатную вилку в пределах от 10 до 100 тыс. руб. Еще до 200 тыс. руб. и далее до 300 тыс. руб. наблюдается небольшая резко уменьшающаяся доля вакансий. Далее редкие отдельные «пеньки» от нескольких до единичных вакансий.
- более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится около 19 %. Вахтовый метод примерно 0.8 %. Относительно формата занятости: более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %.
- требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. При этом половина устанавливает опыт от 1 до 3 лет, почти 30 % от 3 до 6 лет и 2.7 % - больше 6 лет опыта. Опыт прямо связан зарплатными предложениями, при росте опыта растет предлагаемая зарплата.
- если посмотреть количество регионов для каждого работодателя, в которых он публикует свои вакансии, то мы видим корреляцию с количеством вакансий у работодателей, т.е. при росте числа вакансий растет число регионов публикации.
- почти 8.5 тыс. работодателей не указали сферы деятельности в которых они предлагают работу, это без малого 36 %.
- связь между количеством ключевых навыков и зарплатой отсутствует. Ключевых навыков указано от 1 до 30. При этом в 17.2 % вакансий ключевые навыки не указаны.

Анализ вакансий, связанных с «Data Science»:
- 1771 вакансия имеет отношение к данным (в названии присутствуют или «data» или «данн»), то есть 3.6 % от всего количества вакансий. Если говорить о вакансиях для дата-сайентистов (исходя из перечня указанных условий для названий), то таковых 480 или чуть меньше 1 %.
- всего в 51 вакансии указан уровень Junior, это 10.6 % от всех вакансий, для дата-сайентистов.
- SQL или postgres в качестве ключевых навыков указаны в 201 вакансии для дата-сайентистов). Python популяренее – присутствует в 351 вакансии, что составляет 73 % всех вакансий для дата-сайентистов.
- 6.41 среднее количество ключевых навыков в вакансиях для дата-сайентистов.

    Выводы:
Специальность «Data Science» достаточно редко востребована в общем количестве вакансий ИТ сферы. Вакансии предлагаются скорее крупными компаниями (банки и иные финансовые компании, телекоммуникационные корпорации, интернет-гиганты торговой, социально-сетевой, игровой и т.п. направленности, торговые гиганты и т.п.), крупные промышленные компании и научные учреждения. Востребованы специалисты с опытом, с набором определенных ключевых навыков. Вакансии предлагаются в крупных городах, Москве, СПб. Возможная удаленная работа. Заработная плата в целом в два – три раза выше, чем в среднем для ИТ сферы. И зависит в первую очередь от опыта.ию

# Дополнительные исследования

Получим информацию о сбербанке:
- из какого региона этот работодатель,
- в каких сферах деятельности он предлагает вакансии,
- в каких регионах предлагаются вакансии,
- какие именно вакансии (название, график, оплата, регион вакансии и т.д.).

In [388]:
# текст запроса
SBER_1 = f'''SELECT 
                    e.id id_eployer,
                    e.name,
                    a.name area
                FROM 
                    employers e
                    JOIN areas a ON e.area = a.id -- присоединяем только для имени региона
                WHERE 
                    e.id = 3529 -- мы знаем, что это id СБЕРБАНК
             '''

In [387]:
# результат запроса
sber = pd.read_sql_query(SBER_1, connection)
sber

,id_eployer,name,area
0,3529,СБЕР,Москва


In [399]:
# текст запроса
SBER_2 = f'''SELECT 
                    COUNT(*) industries
                FROM 
                    employers_industries e_i
                    LEFT JOIN industries i ON e_i.industry_id = i.id -- присоединяем для вывода названий сфер деятельности
                WHERE 
                    e_i.employer_id = 3529 -- id СБЕР
             '''

In [394]:
sber_industries = pd.read_sql_query(SBER_2, connection)
sber_industries

,industries
0,1


In [391]:
SBER_3 = f'''SELECT 
                    a.name area_vacancy,
                    COUNT(*) quantity
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- добавляем названия регионов
                WHERE 
                    v.employer_id = 3529 
                GROUP BY a.name -- сгруппируем по названию региона
                ORDER BY quantity DESC, area_vacancy -- отсортируем по количеству вакансий, ни и по названию региона по алфавиту 
             '''

In [395]:
sber_areas = pd.read_sql_query(SBER_3, connection)
sber_areas

,area_vacancy,quantity
0,Москва,205
1,Нижний Новгород,30
2,Новосибирск,30
3,Самара,29
4,Екатеринбург,22
5,Санкт-Петербург,20
6,Иннополис,19
7,Рязань,14
8,Ростов-на-Дону,11
9,Хабаровск,11


In [396]:
# текст запроса
SBER_4 = f'''SELECT 
                    v.name,
                    v.key_skills,
                    v.schedule,
                    v.experience,
                    v.employment,
                    v.salary_from,
                    v.salary_to,
                    a.name area_vacancy
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- просто добавляем нормальное название региона вакансии
                WHERE 
                    v.employer_id = 3529 
                ORDER BY  v.salary_from DESC, v.salary_to DESC -- тут можно по-всякому сортировать и выводить по кусочкам, для удобства анализа
             '''

In [397]:
sber_vacancies = pd.read_sql_query(SBER_4, connection)
sber_vacancies

,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_vacancy
0,Системный аналитик DWH,Atlassian Jira\tAtlassian Confluence\tSQL\tScr...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,Москва
1,Стажер Data Engineer/Data Аналитик в команду Б...,Python\tSQL\tMS PowerPoint,Полный день,Нет опыта,Стажировка,NaN,NaN,Москва
2,Junior Java-разработчик,None,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,Москва
3,Java-разработчик,None,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,Ростов-на-Дону
4,Старший инженер технической поддержки автомати...,Linux\tОбучение и развитие\tMac Os\tАвтоматизи...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,NaN,Самара
...,...,...,...,...,...,...,...,...
423,JavaScript Junior Сценарист/разработчик,JavaScript,Полный день,Нет опыта,Полная занятость,60000.0,NaN,Омск
424,Специалист по технической поддержке продуктов ...,None,Полный день,Нет опыта,Полная занятость,50000.0,NaN,Абакан
425,Специалист технической поддержки,None,Гибкий график,От 1 года до 3 лет,Полная занятость,43600.0,NaN,Тольятти
426,Аналитик данных (старший аудитор),None,Полный день,От 1 года до 3 лет,Полная занятость,40800.0,NaN,Нижний Новгород


выводы.
- Сбер как ему и положено находится в Москве.
- сфера как и предпологалась чистый финтех
- а вот регионов како то маловато , Россия ведь большая, ну и как всегда вся работа находиться в Москве.
- всего вакансий у сбера 428
 

In [398]:
# The end...
connection.close()